# Sentiment Index Notebook

## Imports

In [ ]:
%pip install finnhub-python


In [ ]:
import os
import json
import time
import hashlib
from dataclasses import dataclass
from typing import Any, Dict, List

import numpy as np
import pandas as pd
import requests
import yfinance as yf
import matplotlib.pyplot as plt


## Config

In [ ]:
SYMBOL = "AAPL"
START = "2024-01-01"
END   = "2024-06-30"

# FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY", "").strip()
FINNHUB_API_KEY = "d5lbg61r01qgquflgn9gd5lbg61r01qgquflgna0"
assert FINNHUB_API_KEY, "Set FINNHUB_API_KEY as an environment variable first."

CACHE_DIR = "cache/finnhub"
os.makedirs(CACHE_DIR, exist_ok=True)

MIN_INTERVAL_S = 1.2   # safe pacing for free tier
TIMEOUT_S = 20


## Finnhub client

In [ ]:
# @dataclass
# class FinnhubClient:
#     api_key: str
#     cache_dir: str = CACHE_DIR
#     min_interval_s: float = MIN_INTERVAL_S
#     timeout_s: float = TIMEOUT_S

#     def __post_init__(self):
#         self._last_call_ts = 0.0

#     def _sleep_if_needed(self):
#         dt = time.time() - self._last_call_ts
#         if dt < self.min_interval_s:
#             time.sleep(self.min_interval_s - dt)

#     def _cache_path(self, url: str, params: Dict[str, Any]) -> str:
#         raw = url + "?" + "&".join(f"{k}={params[k]}" for k in sorted(params.keys()))
#         h = hashlib.sha256(raw.encode("utf-8")).hexdigest()[:24]
#         return os.path.join(self.cache_dir, f"{h}.json")

#     def _get_json(self, url: str, params: Dict[str, Any], use_cache: bool = True) -> Any:
#         params = dict(params)
#         params["token"] = self.api_key

#         cache_path = self._cache_path(url, params)
#         if use_cache and os.path.exists(cache_path):
#             with open(cache_path, "r", encoding="utf-8") as f:
#                 return json.load(f)

#         self._sleep_if_needed()
#         r = requests.get(url, params=params, timeout=self.timeout_s)
#         self._last_call_ts = time.time()

#         if r.status_code == 429:
#             raise RuntimeError("Finnhub rate limit hit (429). Reduce calls or rely on cache.")

#         r.raise_for_status()
#         data = r.json()

#         if use_cache:
#             with open(cache_path, "w", encoding="utf-8") as f:
#                 json.dump(data, f)

#         return data

#     def company_news(self, symbol: str, date_from: str, date_to: str) -> List[Dict[str, Any]]:
#         url = "https://finnhub.io/api/v1/company-news"
#         params = {"symbol": symbol, "from": date_from, "to": date_to}
#         print("params: ", params)
#         data = self._get_json(url, params=params, use_cache=True)
#         print("data: ", data)
#         return list(data) if isinstance(data, list) else []


## Sentiment index

In [ ]:
POS_WORDS = {
    "beat", "beats", "growth", "surge", "record", "profit", "profits", "upgrade",
    "strong", "bull", "bullish", "outperform", "rally", "upside", "win", "wins"
}
NEG_WORDS = {
    "miss", "misses", "drop", "falls", "plunge", "loss", "losses", "downgrade",
    "weak", "bear", "bearish", "underperform", "crash", "downside", "lawsuit"
}

def _tokenize(text: str) -> List[str]:
    if not text:
        return []
    t = "".join(ch.lower() if ch.isalnum() else " " for ch in text)
    return [w for w in t.split() if len(w) >= 2]

def headline_polarity(headline: str) -> float:
    toks = _tokenize(headline)
    if not toks:
        return 0.0
    pos = sum(1 for w in toks if w in POS_WORDS)
    neg = sum(1 for w in toks if w in NEG_WORDS)
    return float((pos - neg) / (pos + neg + 1e-9))


## Daily prices (yFinance)

In [ ]:
df = yf.download(SYMBOL, start=START, end=END, interval="1d", auto_adjust=True, progress=False)
df = df.dropna().copy()
df.index = pd.to_datetime(df.index).tz_localize(None)  # keep dates naive
df["date"] = df.index.strftime("%Y-%m-%d")
df.head()


## Raw Sentiment index

In [ ]:
# client = FinnhubClient(api_key=FINNHUB_API_KEY)

from data.finnhub_api import FinnhubClient

client = FinnhubClient(api_key=FINNHUB_API_KEY)

news = client.company_news_range(
    symbol="AAPL",
    date_from="2024-01-02",
    date_to="2025-01-02"
)

len(news), news[:2]


# dates = df["date"].tolist()
# raw = np.zeros(len(dates), dtype=np.float32)
# n_news = np.zeros(len(dates), dtype=np.int32)
# print("dates")
# print(dates)

# for i, d in enumerate(dates):
#     news = client.company_news(SYMBOL, "2020-06-01", "2020-06-10")
#     # news = client.company_news(SYMBOL, _from="2020-06-01", to="2020-06-10")

#     # print("news")
#     # print(news)
#     n_news[i] = len(news)
#     if not news:
#         raw[i] = 0.0
#         continue
#     pols = [headline_polarity(item.get("headline") or "") for item in news]
#     raw[i] = float(np.mean(pols)) if pols else 0.0

# df["news_count"] = n_news
# df["sent_raw"] = raw
# df[["Close", "news_count", "sent_raw"]].head(10)


## Score

In [ ]:
def ewma(x: np.ndarray, alpha: float = 0.2) -> np.ndarray:
    out = np.zeros_like(x, dtype=np.float32)
    m = 0.0
    for i in range(len(x)):
        m = alpha * float(x[i]) + (1.0 - alpha) * m
        out[i] = m
    return out

def rolling_z(x: np.ndarray, window: int = 60, clip: float = 3.0) -> np.ndarray:
    out = np.zeros_like(x, dtype=np.float32)
    for i in range(len(x)):
        lo = max(0, i - window + 1)
        win = x[lo:i+1]
        mu = float(np.mean(win))
        sd = float(np.std(win)) + 1e-6
        z = (float(x[i]) - mu) / sd
        if clip is not None:
            z = float(np.clip(z, -clip, clip))
        out[i] = z
    return out

df["sent_ewma"] = ewma(df["sent_raw"].values, alpha=0.2)
df["sent_z"] = rolling_z(df["sent_ewma"].values, window=60, clip=3.0)

df[["news_count","sent_raw","sent_ewma","sent_z"]].describe()


## Debug

In [ ]:
print("Days:", len(df))
print("Days with news:", int((df["news_count"] > 0).sum()))
print("Raw sentiment min/mean/max:", float(df["sent_raw"].min()), float(df["sent_raw"].mean()), float(df["sent_raw"].max()))
print("Z sentiment min/mean/max:", float(df["sent_z"].min()), float(df["sent_z"].mean()), float(df["sent_z"].max()))

# If 'Days with news' is too low, widen date range or switch API later.


## Testing Alpha Vanatage

In [ ]:
from data.finnhub_api import AlphaVantageNewsClient

client = AlphaVantageNewsClient(
    api_key="FYFJWZUN7XRMOHJT",
    cache_dir="cache/alphavantage_news",
)

tickers = ["AAPL", "AMZN", "MSFT", "GOOGL", "META"]
start_date = "2018-01-01"
end_date = "2024-01-01"

all_news = {}
for t in tickers:
    all_news[t] = client.fetch_news(t, start_date, end_date, limit=1000, use_cache=True)


Fetching data for:  {'function': 'NEWS_SENTIMENT', 'tickers': 'AAPL', 'time_from': '20230101T0000', 'limit': '1000', 'sort': 'LATEST', 'apikey': 'FYFJWZUN7XRMOHJT'}
Fetching data for:  {'function': 'NEWS_SENTIMENT', 'tickers': 'AAPL', 'time_from': '20230101T0000', 'limit': '1000', 'sort': 'LATEST', 'apikey': 'FYFJWZUN7XRMOHJT'}
Fetching data for:  {'function': 'NEWS_SENTIMENT', 'tickers': 'AAPL', 'time_from': '20230101T0000', 'limit': '1000', 'sort': 'LATEST', 'apikey': 'FYFJWZUN7XRMOHJT'}
Fetching data for:  {'function': 'NEWS_SENTIMENT', 'tickers': 'AAPL', 'time_from': '20230101T0000', 'limit': '1000', 'sort': 'LATEST', 'apikey': 'FYFJWZUN7XRMOHJT'}
Fetching data for:  {'function': 'NEWS_SENTIMENT', 'tickers': 'AMZN', 'time_from': '20230101T0000', 'limit': '1000', 'sort': 'LATEST', 'apikey': 'FYFJWZUN7XRMOHJT'}
Fetching data for:  {'function': 'NEWS_SENTIMENT', 'tickers': 'AMZN', 'time_from': '20230101T0000', 'limit': '1000', 'sort': 'LATEST', 'apikey': 'FYFJWZUN7XRMOHJT'}
Fetching d